In [36]:
import pandas as pd
import numpy as np

df = pd.read_csv('data-cleaned.csv')
label=df["isNaturalCaused"]
df.drop(labels=["isNaturalCaused","fire_year", "true_cause", "general_cause_desc"], axis=1, inplace=True)
#df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

#Preprocessing:

categorical = [var for var in df.columns if df[var].dtype=='O']
numerical = [var for var in df.columns if var not in categorical]

In [37]:
# clean up categorical null variable
for var in categorical:
  df[var]= df[var].fillna("N/A")
  print(df[var].value_counts())


# clean up numerical null variable using median
for var in numerical:
  df[var] = df[var].fillna(df[var].median())

df.isnull().sum()

Surface       16718
Ground         5022
N/A            2569
Crown          1010
   Surface        1
                  1
Name: fire_type, dtype: int64
Flat          18384
N/A            2809
Bottom         1388
Upper 1/3      1070
Middle 1/3      910
Lower 1/3       760
Name: fire_position_on_slope, dtype: int64
Clear          10514
Cloudy          8121
N/A             2818
CB Wet          1502
CB Dry          1414
Rainshowers      952
Name: weather_conditions_over_fire, dtype: int64
W      5103
CLM    3249
N/A    2823
SW     2714
NW     2682
SE     2470
E      1820
S      1590
N      1553
NE     1314
 S        2
 NW       1
Name: wind_direction, dtype: int64


assessment_hectares             0
current_size                    0
fire_spread_rate                0
fire_type                       0
fire_position_on_slope          0
weather_conditions_over_fire    0
temperature                     0
relative_humidity               0
wind_direction                  0
wind_speed                      0
dtype: int64

In [38]:
# Apply Labeling Encoding
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
for var in categorical:
  df[var] = le.fit_transform(df[var])

df.head()

,assessment_hectares,current_size,fire_spread_rate,fire_type,fire_position_on_slope,weather_conditions_over_fire,temperature,relative_humidity,wind_direction,wind_speed
0,0.01,0.10,0.0,5,1,2,18.0,10.0,10,2.0
1,0.20,0.20,0.0,5,2,2,12.0,22.0,10,10.0
2,0.50,0.50,0.0,5,0,2,12.0,22.0,10,10.0
3,0.01,0.01,0.0,5,1,2,12.0,22.0,10,10.0
4,0.10,0.10,0.1,5,1,2,6.0,37.0,10,2.0


In [39]:
# handling imbalanced data
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

x = df
y = label

smk = SMOTETomek()
x_res,y_res = smk.fit_resample(x,y)

x_res.shape, y_res.shape

((32076, 10), (32076,))

In [40]:
from sklearn.model_selection import train_test_split
import random

# train test split
x_train, x_test, y_train, y_test = train_test_split(x_res,y_res,test_size=0.20,shuffle=True)

print("X_train shape:",x_train.shape)
print("X_test shape:",x_test.shape)
print("y_train shape:",y_train.shape)
print("y_test shape:",y_test.shape)

X_train shape: (25660, 10)
X_test shape: (6416, 10)
y_train shape: (25660,)
y_test shape: (6416,)


In [41]:
import sklearn.tree
import time

decision_tree = sklearn.tree.DecisionTreeClassifier(criterion = "log_loss")

start_time = time.time()
decision_tree.fit(x_train, y_train)
end_time = time.time()
elapsed_time = end_time - start_time

In [42]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

grid = {
    'criterion' : ["gini","entropy","log_loss"]
}

tree  = sklearn.tree.DecisionTreeClassifier()
tree_cv = GridSearchCV(tree, grid, cv=5)
tree_cv.fit(x_train,y_train)
print("Best Parameters:",tree_cv.best_params_)
pred_y = tree_cv.predict(x_test)

cm = metrics.confusion_matrix(y_test, pred_y)
cm_df = pd.DataFrame(cm, index=["Actual Human Caused", "Actual Natural Caused"], columns=["Predicted Human Caused", "Predicted Natural Caused"])

print("Train Score:",tree_cv.best_score_)
print("Test Score:",tree_cv.score(x_test,y_test))

print("\nDecision Tree")
print("Accuracy:", metrics.accuracy_score(y_test, pred_y))
print("Precision:", metrics.precision_score(y_test, pred_y))
print("\n", cm_df)
print("\n",metrics.classification_report(y_test, pred_y, digits=6))
print(f"Fitting Time: {elapsed_time}s")

Best Parameters: {'criterion': 'log_loss'}
Train Score: 0.8386204208885424
Test Score: 0.8435162094763092

Decision Tree
Accuracy: 0.8435162094763092
Precision: 0.8444931163954944

                        Predicted Human Caused  Predicted Natural Caused
Actual Human Caused                      2713                       497
Actual Natural Caused                     507                      2699

               precision    recall  f1-score   support

           0   0.842547  0.845171  0.843857      3210
           1   0.844493  0.841859  0.843174      3206

    accuracy                       0.843516      6416
   macro avg   0.843520  0.843515  0.843515      6416
weighted avg   0.843519  0.843516  0.843516      6416

Fitting Time: 0.07450580596923828s
